In [1]:
# set up logging
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

In [2]:
# make deterministic
from mingpt.utils import set_seed
set_seed(42)

In [3]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
import random
import os

In [8]:
# config
numEpochs = 50 # number of epochs to train the GPT+PT model
embeddingSize = 512 # the hidden dimension of the representation of both GPT and PT
numPoints=500 # number of points that we are going to receive to make a prediction about f given x and y, if you don't know then use the maximum
numVars=5 # the dimenstion of input points x, if you don't know then use the maximum
numYs=1 # the dimension of output points y = f(x), if you don't know then use the maximum
blockSize = 100 # spatial extent of the model for its context
batchSize = 128 # batch size of training data
dataDir = 'D:/Datasets/Symbolic Dataset/Datasets/FirstDataGenerator'
dataInfo = 'XYE_{}Var_{}Points_{}EmbeddingSize'.format(numVars, numPoints, embeddingSize)
target = 'Skeleton' #'Skeleton' #'EQ'
dataFolder = "1-5Var_RandSupport_RandLength_0to3_3.1to6_100to500Points"
addr = './SavedModels/bestModel/' # where to save model
maxNumFiles = 30
bestLoss = None # if there is any model to load as pre-trained one

In [9]:
import math
from torch.utils.data import Dataset

class CharDataset(Dataset):

    def __init__(self, data, block_size, chars, target='EQ'):
        data_size, vocab_size = len(data), len(chars)
        print('data has %d examples, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        
        # padding token
        self.paddingToken = '_'
        self.paddingID = self.stoi[self.paddingToken]
        self.stoi[self.paddingToken] = self.paddingID
        self.itos[self.paddingID] = self.paddingToken
        self.threshold = [-1000,1000]
        
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data # it should be a list of examples
        self.target = target
    
    def __len__(self):
        return len(self.data)-1

    def __getitem__(self, idx):
        # grab an example from the data
        chunk = self.data[idx] # sequence of tokens including x, y, eq, etc.
        
        try:
            chunk = json.loads(chunk) # convert the sequence tokens to a dictionary
        except:
            print("Couldn't convert to json: {}".format(chunk))
            
        # encode every character in the equation to an integer
        # < is SOS, > is EOS
        dix = [self.stoi[s] for s in '<'+chunk[self.target]+'>']
        inputs = dix[:-1]
        outputs = dix[1:]
        
        # add the padding to the equations
        paddingSize = max(self.block_size-len(inputs),0)
        paddingList = [self.paddingID]*paddingSize
        inputs += paddingList
        outputs += paddingList 
        
        # make sure it is not more than what should be
        inputs = inputs[:self.block_size]
        outputs = outputs[:self.block_size]
        
        # extract points from the input sequence
        points = torch.zeros(numVars+numYs, numPoints)
        for idx, xy in enumerate(zip(chunk['X'], chunk['Y'])):
            x = xy[0] + [0]*(max(numVars-len(xy[0]),0)) # padding
            y = [xy[1]] if type(xy[1]) == float or type(xy[1]) == int else xy[1]
            y = y + [0]*(max(numYs-len(y),0)) # padding
            p = x+y # because it is only one point 
            p = torch.tensor(p)
            #replace nan and inf
            p = torch.nan_to_num(p, nan=0.0, 
                                 posinf=self.threshold[1], 
                                 neginf=self.threshold[0])
            p[p>self.threshold[1]] = self.threshold[1] # clip the upper bound
            p[p<self.threshold[0]] = self.threshold[0] # clip the lower bound
            points[:,idx] = p
        
        """
        arrange data and targets so that the first i elements of x
        will be asked to predict the i-th element of y. Notice that
        the eventual language model will actually make block_size
        individual predictions at the same time based on this data,
        so we are being clever and amortizing the cost of the forward
        pass of the network. So for example if block_size is 4, then
        we could e.g. sample a chunk of text "hello", the integers in
        x will correspond to "hell" and in y will be "ello". This will
        then actually "multitask" 4 separate examples at the same time
        in the language model:
        - given just "h", please predict "e" as next
        - given "he" please predict "l" next
        - given "hel" predict "l" next
        - given "hell" predict "o" next
        
        In addition, because the DataLoader will create batches of examples,
        every forward/backward pass during traning will simultaneously train
        a LOT of predictions, amortizing a lot of computation. In particular,
        for a batched input of integers X (B, T) where B is batch size and
        T is block_size and Y (B, T), the network will during training be
        simultaneously training to make B*T predictions, all at once! Of course,
        at test time we can paralellize across batch B, but unlike during training
        we cannot parallelize across the time dimension T - we have to run
        a forward pass of the network to recover the next single character of the 
        sequence along each batch dimension, and repeatedly always feed in a next
        character to get the next one.
        
        So yes there is a big asymmetry between train/test time of autoregressive
        models. During training we can go B*T at a time with every forward pass,
        but during test time we can only go B at a time, T times, with T forward 
        passes.
        """
        inputs = torch.tensor(inputs, dtype=torch.long)
        outputs = torch.tensor(outputs, dtype=torch.long)
        return inputs, outputs, points


In [10]:
import json
from tqdm import tqdm
import glob
def processDataFiles(path, files):
    text = ''""
    for f in tqdm(files):
        with open(path+f, 'r') as h: 
            lines = h.read() # don't worry we won't run out of file handles
            if lines[-1]==-1:
                lines = lines[:-1]
            text += lines #json.loads(line)       
            
    return text

In [11]:
path = '{}/{}/Train/'.format(dataDir, dataFolder)
files = os.listdir(path)[:maxNumFiles]
text = processDataFiles(path, files)
chars = sorted(list(set(text))+['_','T','<','>']) # extract unique characters from the text before converting the text to a list
# T is for the test data
text = text.split('\n') # convert the raw text to a set of examples
text = text[:-1] if len(text[-1]) == 0 else text
random.shuffle(text) # shuffle the dataset, it's important for combined number of variables
train_dataset = CharDataset(text, blockSize, chars, target=target) 

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [24:47<00:00, 49.59s/it]


data has 300000 examples, 48 unique.


In [12]:
idx = np.random.randint(train_dataset.__len__())
inputs, outputs, points = train_dataset.__getitem__(idx)
print('inputs:{}'.format(inputs))
inputs = ''.join([train_dataset.itos[int(i)] for i in inputs])
outputs = ''.join([train_dataset.itos[int(i)] for i in outputs])
print('id:{}\ninputs:{}\noutputs:{}\npoints:{}'.format(idx,inputs,outputs,points))

inputs:tensor([21, 43, 36, 39,  3, 23,  5, 45, 13,  6, 23,  4, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32])
id:121958
inputs:<sin(C*x3+C)________________________________________________________________________________________
outputs:sin(C*x3+C)>________________________________________________________________________________________
points:tensor([[ 2.4800,  0.2400,  1.2500,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.4200,  0.7200,  2.7600,  ...,  0.0000,  0.0000,  0.0000],
        [ 2.7700,  1.5800,  0.2500,  ...,  0.0000,  0.0000,  0.0000],
        [ 2.0100,  2.6300,  2.0800,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.2800,  1.1500,  1.

In [13]:
path = '{}/{}/Val/'.format(dataDir,dataFolder)
files = os.listdir(path)[:maxNumFiles]
textVal = processDataFiles(path, files)
textVal = textVal.split('\n') # convert the raw text to a set of examples
val_dataset = CharDataset(textVal, blockSize, chars, target=target)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 78.30it/s]

data has 1001 examples, 48 unique.


In [14]:
idx = np.random.randint(val_dataset.__len__())
inputs, outputs, points = val_dataset.__getitem__(idx)
print(points.min(), points.max())
inputs = ''.join([train_dataset.itos[int(i)] for i in inputs])
outputs = ''.join([train_dataset.itos[int(i)] for i in outputs])
print('id:{}\ninputs:{}\noutputs:{}\npoints:{}'.format(idx,inputs,outputs,points))

tensor(0.) tensor(3.4900)
id:435
inputs:<C*x2+C_____________________________________________________________________________________________
outputs:C*x2+C>_____________________________________________________________________________________________
points:tensor([[0.2800, 2.5100, 0.4200,  ..., 0.0000, 0.0000, 0.0000],
        [1.5900, 1.1100, 1.9100,  ..., 0.0000, 0.0000, 0.0000],
        [1.5700, 1.4800, 2.4000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [2.0900, 1.6100, 2.4100,  ..., 0.0000, 0.0000, 0.0000]])


In [16]:
path = '{}/{}/Test/'.format(dataDir,dataFolder)
files = os.listdir(path)[:maxNumFiles]
textTest = processDataFiles(path, files)
textTest = textTest.split('\n') # convert the raw text to a set of examples
# test_dataset_target = CharDataset(textTest, blockSize, chars, target=target)
test_dataset = CharDataset(textTest, blockSize, chars)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 91.11it/s]

data has 1001 examples, 48 unique.


In [17]:
idx = np.random.randint(test_dataset.__len__())
inputs, outputs, points = test_dataset.__getitem__(idx)
print(points.min(), points.max())
inputs = ''.join([train_dataset.itos[int(i)] for i in inputs])
outputs = ''.join([train_dataset.itos[int(i)] for i in outputs])
print('id:{}\ninputs:{}\noutputs:{}\npoints:{}'.format(idx,inputs,outputs,points))

tensor(0.) tensor(3.)
id:860
inputs:<0.14*x3+sqrt(abs(-0.93*x5))________________________________________________________________________
outputs:0.14*x3+sqrt(abs(-0.93*x5))>________________________________________________________________________
points:tensor([[2.8800, 1.3000, 0.6400,  ..., 0.0000, 0.0000, 0.0000],
        [0.7300, 1.1300, 0.6500,  ..., 0.0000, 0.0000, 0.0000],
        [1.1000, 2.9600, 0.5200,  ..., 0.0000, 0.0000, 0.0000],
        [2.9100, 0.9100, 2.8700,  ..., 0.0000, 0.0000, 0.0000],
        [0.3600, 1.4800, 2.3100,  ..., 0.0000, 0.0000, 0.0000],
        [0.7300, 1.5800, 1.5400,  ..., 0.0000, 0.0000, 0.0000]])


In [23]:
from mingpt.model import GPT, GPTConfig, PointNetConfig
pconf = PointNetConfig(embeddingSize=embeddingSize, 
                       numberofPoints=numPoints, 
                       numberofVars=numVars, 
                       numberofYs=numYs)
mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size,
                  n_layer=8, n_head=8, n_embd=embeddingSize, padding_idx=train_dataset.paddingID)
model = GPT(mconf, pconf)

05/26/2021 10:13:34 - INFO - mingpt.model -   number of parameters: 3.058177e+07


In [19]:
fName = '{}_SymbolicGPT_{}_{}_{}_MINIMIZE.txt'.format(dataInfo, 
                                             'GPT_PT_Summation', 
                                             'Padding',
                                             blockSize)

In [20]:
ckptPath = '{}/{}.pt'.format(addr,fName.split('.txt')[0])
print(ckptPath)
if bestLoss != None:
    # load the best model
    print('Model has been loaded!')
    model.load_state_dict(torch.load(ckptPath))

./SavedModels/bestModel//XYE_5Var_500Points_512EmbeddingSize_SymbolicGPT_GPT_PT_Summation_Padding_100_MINIMIZE.pt


In [24]:
from mingpt.trainer import Trainer, TrainerConfig
import os

try: 
    os.mkdir(addr)
except:
    print('Folder already exists!')
    
# initialize a trainer instance and kick off training
tconf = TrainerConfig(max_epochs=numEpochs, batch_size=batchSize, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=512*20, final_tokens=2*len(train_dataset)*blockSize,
                      num_workers=0, ckpt_path=ckptPath)
trainer = Trainer(model, train_dataset, val_dataset, tconf, bestLoss)

try:
    trainer.train()
except KeyboardInterrupt:
    print('KeyboardInterrupt')

  0%|                                                                                         | 0/2344 [00:00<?, ?it/s]

Folder already exists!


epoch 1 iter 1048: train loss 0.13302. lr 5.289431e-04:  45%|█████████▊            | 1049/2344 [42:20<52:10,  2.42s/it]

Input:tensor([21, 43, 36, 39,  3, 43, 36, 39,  3, 23,  5, 45, 11,  6, 23,  4,  4, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32], device='cuda:0')
Logit:tensor([43, 36, 39,  3, 23, 36, 39,  3, 23,  5, 45, 11,  6, 23,  4,  4, 22,  5,
         5, 12, 12,  5, 44,  5, 22, 22, 43,  5, 22, 22, 12,  6, 43,  5, 39, 23,
        45,  6,  6,  5, 22, 39, 22, 22, 23,  5, 22,  6, 39, 22, 43, 22, 39,  5,
        22,  5,  5,  6,  5,  5,  4,  4,  5,  5,  5,  5,  5,  4,  6, 43,  5,  5,
         6,  5,  6,  4, 23,  5,  4,  5, 22,  5,  5,  5,  5,  5, 43,  5, 43, 22,
         3, 22,  6, 43,  5, 23,  5,  6,  5, 22], device='cuda:0')
Input:<sin(sin(C*x1+C))_________________________________

epoch 1 iter 1471: train loss 0.10722. lr 4.655640e-04:  63%|█████████████▊        | 1472/2344 [59:22<34:01,  2.34s/it]

...

Input:tensor([21, 43, 41, 42, 44,  3, 33, 34, 43,  3, 23,  5, 45, 12,  6, 23,  5, 45,
        13,  6, 12,  5, 23,  4,  4, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32], device='cuda:0')
Logit:tensor([43, 41, 42, 44,  3, 33, 34, 43,  3, 23,  5, 45, 13,  6, 23,  5, 45, 13,
         6, 12,  5, 23,  4,  4, 22, 43, 43, 34, 43, 44, 14,  6, 34, 45,  4,  6,
         6, 23, 43, 43, 14,  4,  6, 22, 33,  6, 12,  6,  4,  6, 34,  6, 34,  6,
        44, 23, 43,  6,  5, 13, 12,  6, 43, 43,  6,  5,  4,  6, 43, 41,  6,  6,
         6,  6,  5, 45,  6,  5,  4,  5,  6, 43,  6,  6,  6, 43, 43,  6, 41, 22,
        41,  6,  6, 43,  5, 43,  5,  6,  5, 22], device='cuda:0')
Input:<sqrt(abs(C*x2+C*x3+2*C))_________________________

epoch 1 iter 2343: train loss 0.09756. lr 3.000804e-04: 100%|████████████████████| 2344/2344 [1:34:31<00:00,  2.42s/it]
05/26/2021 11:48:25 - INFO - mingpt.trainer -   test loss: 0.176251
05/26/2021 11:48:25 - INFO - mingpt.trainer -   saving ./SavedModels/bestModel//XYE_5Var_500Points_512EmbeddingSize_SymbolicGPT_GPT_PT_Summation_Padding_100_MINIMIZE.pt
epoch 2 iter 507: train loss 0.07408. lr 1.998815e-04:  22%|████▊                 | 508/2344 [20:29<1:15:29,  2.47s/it]

Input:tensor([21, 23,  5,  5, 12,  5, 45, 11,  5,  5, 12,  6, 12,  5, 23,  5,  5, 12,
         5, 45, 11,  6, 23,  5,  5, 12,  6, 23,  5, 45, 11,  6, 23, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32], device='cuda:0')
Logit:tensor([23,  5, 45, 12,  5, 45, 11,  5,  5, 12,  6, 12,  5, 23,  5,  5, 12,  5,
        45, 11,  6, 23,  5,  5, 12,  6, 12,  5, 45, 11,  6, 23, 22,  6,  6,  6,
        45,  6,  6, 45, 39,  6,  6, 22, 23, 12, 43, 34, 23, 45,  6,  6,  6, 43,
        44, 23, 22, 23, 43,  5, 22,  3, 23, 23, 23, 34, 12, 43,  6, 43,  6, 34,
         3,  5, 22, 22, 34, 23,  6, 23,  6, 23,  6,  3,  6, 23, 34, 34, 43,  5,
        22, 34, 34,  6, 23, 34, 43,  5, 23,  6], device='cuda:0')
Input:<C**2*x1**2+2*C**2*x1+C**2+C*x1+C_________________

epoch 2 iter 522: train loss 0.07415. lr 1.970431e-04:  22%|████▉                 | 523/2344 [21:08<1:13:35,  2.42s/it]

KeyboardInterrupt


In [25]:
# load the best model
model.load_state_dict(torch.load(ckptPath))
model = model.eval().to(trainer.device)

In [26]:
# add a safe wrapper for numpy math functions
from numpy import *
import numpy as np

def divide(x, y):
  x = np.nan_to_num(x)
  y = np.nan_to_num(y)
  return np.divide(x,y+1e-5)

def sqrt(x):
  x = np.nan_to_num(x)
  return np.sqrt(np.abs(x)) 

# Mean square error
def mse(y, y_hat):
    y_hat = np.reshape(y_hat, [1, -1])[0]
    y_gold = np.reshape(y, [1, -1])[0]
    our_sum = 0
    for i in range(len(y_gold)):
        our_sum += (y_hat[i] - y_gold[i]) ** 2

    return our_sum / len(y_gold)

# Mean square error
def relativeErr(y, y_hat):
    y_hat = np.reshape(y_hat, [1, -1])[0]
    y_gold = np.reshape(y, [1, -1])[0]
    our_sum = 0
    for i in range(len(y_gold)):
        if y_gold[i] < 1: 
            # use regular MSE
            our_sum += (y_hat[i] - y_gold[i]) ** 2
        else:
            # use relative MSE
            our_sum += ((y_hat[i] - y_gold[i])/y_gold[i]) ** 2

    return our_sum / len(y_gold)

In [27]:
# class pointsDataset(Dataset):

#     def __init__(self, data):
#         # data should be a list of x,y pairs
#         self.x = data[0] # it should be a list
#         self.y = data[1] # it should be a list
#         self.threshold = [-1000,1000]
    
#     def __len__(self):
#         return len(self.y)

#     def __getitem__(self, idx):
#         # grab an example from the data
#         x = self.x[idx] 
#         y = self.y[idx] 
        
#         p = x+[y]
        
#         p = torch.tensor(p, dtype=torch.float)
#         p = torch.nan_to_num(p, nan=0.0, 
#                              posinf=self.threshold[1], 
#                              neginf=self.threshold[0])
#         p[p>self.threshold[1]] = self.threshold[1] # clip the upper bound
#         p[p<self.threshold[0]] = self.threshold[0] # clip the lower bound
        
#         return p
    
# # train a mlp to find the constants
# data = pointsDataset((t['X'],t['Y']))
# loader = torch.utils.data.DataLoader(
#                                 data, 
#                                 shuffle=False, 
#                                 pin_memory=True,
#                                 batch_size=batchSize,
#                                 num_workers=0)
# class MLP(nn.Module):
#     def __init__(self, inputSize, outputSize):
#         super().__init__()
#         self.layers = nn.Sequential(
#             nn.Linear(inputSize, 100),
#             nn.ReLU(),
#             nn.Linear(100, outputSize)
#         )

#     def forward(self, x):
#         c = self.layers(x)
#         return c
    
#     def loss(self, constants, eq, X, Y):
#         # constants is the output of the model
#         err = torch.zeros(len(constants)) # batch_size
        
#         # sample number of points
#         indexes = []
#         numberSamples = 10
#         while len(indexes) != numberSamples:
#             randNum = np.random.randint(len(batch))
#             indexes.append(randNum)
#         X = X[indexes,:]
#         Y = Y[indexes,:]
        
#         # replace the constants with their predicted values
#         for idx, const in enumerate(constants):
#             eq = eq.replace('C','{}').format(*const.tolist())
        
#         # calculate the error for a limited number of points, approximate the error
#         for x,y in zip(X,Y):
#             # replace variables with their values
#             for i,e in enumerate(x):
#                 eqTemp = eq.replace('x{}'.format(i+1), str(e.item()))
                
#             # calculate the error
#             yHat = eval(eqTemp)
#             err[idx] += (y.item()-yHat)**2
#         err[idx] /= numberSamples
            
#         print(err.shape, constants.shape)
#         return err
    
# c = [0 for i,x in enumerate(predicted) if x=='C']
# network = MLP(numVars+numYs, len(c))
# cHat = network(batch)
# err = network.loss(cHat, predicted, batch[:,:numVars], batch[:,-numYs:])

In [ ]:
# alright, let's sample some character-level symbolic GPT
from mingpt.utils import sample
from gp_model import Genetic_Model
from mlp_model import MLP_Model
from scipy.optimize import least_squares, newton, minimize

    
loader = torch.utils.data.DataLoader(
                                test_dataset, 
                                shuffle=False, 
                                pin_memory=True,
                                batch_size=1,
                                num_workers=0)

testRange = [3.1,6.0]
numTestPoints = 10
#test = np.linspace(3.1,6.0,numTestPoints)

gpm = Genetic_Model(n_jobs=-1)
mlp = MLP_Model()

resultDict = {}
try:
    with open(fName, 'w', encoding="utf-8") as o:
        modelName = 'SymbolicGPT'
        resultDict[fName] = {modelName:[],
                             'GP':[],
                             'MLP':[]}

        for i, batch in enumerate(loader):
                
            inputs,outputs,points = batch

            print('Test Case {}.'.format(i))
            o.write('Test Case {}/{}.\n'.format(i,len(textTest)-1))

            t = json.loads(textTest[i])

            inputs = inputs[:,0:1].to(trainer.device)
            points = points.to(trainer.device)
            outputsHat = sample(model, inputs, blockSize, points=points,
                          temperature=1.0, sample=True, 
                          top_k=40)[0]

            # filter out predicted
            target = ''.join([train_dataset.itos[int(i)] for i in outputs[0]])
            predicted = ''.join([train_dataset.itos[int(i)] for i in outputsHat])

            target = target.strip(train_dataset.paddingToken).split('>')
            target = target[0] if len(target[0])>=1 else target[1]
            target = target.strip('<').strip(">")
            predicted = predicted.strip(train_dataset.paddingToken).split('>')
            predicted = predicted[0] if len(predicted[0])>=1 else predicted[1]
            predicted = predicted.strip('<').strip(">")
            
            print('Target:{}\nSkeleton:{}'.format(target, predicted))
            
            o.write('{}\n'.format(target))
            o.write('{}:\n'.format(modelName))
            o.write('{}\n'.format(predicted))

            # train a regressor to find the constants (too slow)
            c = [1 for i,x in enumerate(predicted) if x=='C']            
            def lossFunc(constants, eq, X, Y):
                err = 0
                eq = eq.replace('C','{}').format(*constants)

                for x,y in zip(X,Y):
                    eqTemp = eq + ''
                    for i,e in enumerate(x):
                        eqTemp = eqTemp.replace('x{}'.format(i+1), str(e))
                    try:
                        yHat = eval(eqTemp)
                    except:
                        yHat = 100
                    err += (y-yHat)**2
                err /= len(Y)
                return err
            
            try:
                if len(c) == 0:
                    pass # do nothing
                else:
#                     cHat = least_squares(lossFunc, c, ftol=1e-3,
#                                          args=(predicted, t['X'], t['Y']))
                    
                    # for easier comparison, we are using minimize package  
                    cHat = minimize(lossFunc, c,
                                   args=(predicted, t['X'], t['Y'])) 
            
#                     cHat= newton(lossFunc, c, maxiter=100,
#                                  args=(predicted, t['X'], t['Y']))
                    predicted = predicted.replace('C','{}').format(*cHat.x)
            except:
                print('Wrong Equation:{}'.format(predicted))
                raise
                predicted = 0

            # TODO: let's enjoy GPU

            print('Skeleton+LS:{}'.format(predicted))

            Ys = [] #t['YT']
            Yhats = []
            for xs in t['XT']:
                try:
                    eqTmp = target + '' # copy eq
                    eqTmp = eqTmp.replace(' ','')
                    eqTmp = eqTmp.replace('\n','')
                    for i,x in enumerate(xs):
                        # replace xi with the value in the eq
                        eqTmp = eqTmp.replace('x{}'.format(i+1), str(x))
                        if ',' in eqTmp:
                            assert 'There is a , in the equation!'
                    YEval = eval(eqTmp)
                    YEval = 0 if np.isnan(YEval) else YEval
                    YEval = 100 if np.isinf(YEval) else YEval
                except:
                    YEval = 100 #TODO: Maybe I have to punish the model for each wrong template not for each point
                Ys.append(YEval)
                try:
                    eqTmp = predicted + '' # copy eq
                    eqTmp = eqTmp.replace(' ','')
                    eqTmp = eqTmp.replace('\n','')
                    for i,x in enumerate(xs):
                        # replace xi with the value in the eq
                        eqTmp = eqTmp.replace('x{}'.format(i+1), str(x))
                        if ',' in eqTmp:
                            assert 'There is a , in the equation!'
                    Yhat = eval(eqTmp)
                    Yhat = 0 if np.isnan(Yhat) else Yhat
                    Yhat = 100 if np.isinf(Yhat) else Yhat
                except:
                    Yhat = 100
                Yhats.append(Yhat)
            err = relativeErr(Ys,Yhats)

            if type(err) is np.complex128 or np.complex:
                err = abs(err.real)

            resultDict[fName][modelName].append(err)

            o.write('{}\n{}\n\n'.format( 
                                    predicted,
                                    err
                                    ))

            print('Err:{}'.format(err))
            
            # Calculate error for baselines

    #         # tokenize to get input x, input y, and true eqn
    #         train_data_x = t["X"]
    #         train_data_y = t["Y"]
    #         test_data_x = t["XT"]
    #         test_data_y = t["YT"]

    #         # train MLP model
    #         mlp.reset()
    #         model_eqn, _, best_err = mlp.repeat_train(
    #                                                 train_data_x, 
    #                                                 train_data_y,
    #                                                 test_x=test_data_x, 
    #                                                 test_y=test_data_y,                                     
    #                                                 verbose=False)
    #         test_y_hat = mlp.predict(test_data_x)
    #         err = relativeErr(test_data_y,test_y_hat)
    #         print("{}: {}".format(mlp.name, model_eqn)[:550])
    #         print("Err: {:.5f}".format(err))
    #         resultDict[fName]['MLP'].append(err)
    #         o.write('\n{}: {}\n{}'.format('MLP', 
    #                                    err,
    #                                    model_eqn))

    #         # train GP model
    #         gpm.reset()
    #         model_eqn, _, best_err = gpm.repeat_train(train_data_x, train_data_y,
    #                                                 test_x=test_data_x, test_y=test_data_y,
    #                                                 verbose=False)
    #         print("{}: {}".format(gpm.name, model_eqn)[:550])
    #         test_y_hat = gpm.predict(test_data_x)
    #         err = relativeErr(test_data_y,test_y_hat)
    #         print("Err: {:.5f}".format(err))
    #         resultDict[fName]['GP'].append(err)
    #         o.write('\n{}: {}\n{}'.format('GP', 
    #                                    err,
    #                                    model_eqn))
            print('') # just an empty line
    print('Avg Err:{}'.format(np.mean(resultDict[fName][modelName])))
    
except KeyboardInterrupt:
    print('KeyboardInterrupt')

Test Case 0.
Target:x1-sin(0.64*x1)
Skeleton:2*C**3*x1**3+6*C**3*x1**2+6*C**3*x1+2*C**3
Skeleton+LS:2*-0.0009747521268570955**3*x1**3+6*0.2983095649332288**3*x1**2+6*0.3134883364405661**3*x1+2*0.28204909687529317**3
Err:0.00066794283389326

Test Case 1.
Target:sin(1.27*x1+0.45)
Skeleton:2*C*x1*sin(C*x1+C)+2*C*sin(C*x1+C)
Skeleton+LS:2*0.00019284785188991975*x1*sin(0.06476525574713902*x1+0.8178156763896012)+2*0.4999085917552279*sin(1.2690494279687603*x1+0.45410228865899277)
Err:2.181148389397245e-06

Test Case 2.
Target:-0.33*x1**2*sin(1.0*x1)-0.63*x1
Skeleton:C**2*x1**2*sqrt(abs(C*x1+C))+2*C**2*x1*sqrt(abs(C*x1+C))+C**2*sqrt(abs(C*x1+C))
Skeleton+LS:-0.714111014184743**2*x1**2*sqrt(abs(-2.4361458545299928*x1+13.242824856039677))+2*0.7189734288799192**2*x1*sqrt(abs(4.4559875670153675*x1+-1.3029139272743075))+0.39724910389129525**2*sqrt(abs(0.5066947516830252*x1+-0.06144267557036678))
Err:13.66077803205252

Test Case 3.
Target:1.21*x1+sqrt(abs(0.22-0.58*x1))-0.77
Skeleton:C**2*x1**2+2*C*

Target:2*x1+0.73*sin(0.65*x1+0.23)
Skeleton:2*C*x1+2*C
Skeleton+LS:2*1.0810603173804068*x1+2*0.02166276019706803
Err:0.008651504353931635

Test Case 35.
Target:0.93*x1-0.67
Skeleton:2*C*x1+2*C
Skeleton+LS:2*0.4629904181747298*x1+2*-0.3371934680861228
Err:4.189387698025091e-05

Test Case 36.
Target:sin(0.85*x1-0.64)
Skeleton:sin(C**2*x1**2+2*C**2*x1+C**2)
Skeleton+LS:sin(1.8498950031669008**2*x1**2+2*2.0727570629735363**2*x1+1.806422046001093**2)
Err:0.8700181677886863

Test Case 37.
Target:sin(x1**2+0.48*x1)
Skeleton:sin(C**2*x1**2+2*C**2*x1+C**2)
Skeleton+LS:sin(1.9443660821286894**2*x1**2+2*1.4748780389218512**2*x1+1.5879939656244166**2)
Err:1.0841980607462123

Test Case 38.
Target:0.28*x1**2*sin(x1)-0.26*x1
Skeleton:sin(C**2*x1**2+2*C**2*x1+C**2)
Skeleton+LS:sin(1.8590291280797957**2*x1**2+2*1.4239236652562737**2*x1+1.5016161243087027**2)
Err:36.11796124048018

Test Case 39.
Target:sin(x1-0.16)
Skeleton:sin(C*x1+C)**2
Skeleton+LS:sin(0.7765870485764557*x1+0.21654733138835527)**2
Err

Skeleton+LS:0.6653683130602601*x1*sqrt(abs(0.5868026221801327*x1+0.49677487390285335))+0.04307132626407906*sqrt(abs(0.7719837696787147*x1+0.7291904003401293))
Err:0.0036566623891781643

Test Case 74.
Target:-sin(sin(0.27*x1))
Skeleton:sin(C**2*x1**2+2*C**2*x1+C**2)
Skeleton+LS:sin(1.4330413884869175**2*x1**2+2*2.0099788060071884**2*x1+1.2806313057837437**2)
Err:1.1793383772126904

Test Case 75.
Target:2.1*x1-0.35
Skeleton:2*C**3*x1**3+6*C**3*x1**2+6*C**3*x1+2*C**3
Skeleton+LS:2*0.3576602394078747**3*x1**3+6*0.5954520467358061**3*x1**2+6*0.0005896577176839025**3*x1+2*-0.48327826538117497**3
Err:8.60661006333211

Test Case 76.
Target:-0.33*x1+sqrt(abs(x1))-0.42
Skeleton:sqrt(abs(C*x1+C))+sin(C*x1+C)
Skeleton+LS:sqrt(abs(-0.49591758899425914*x1+0.009971339608356139))+sin(0.23568327733652128*x1+3.401317150150756)
Err:0.14131429164825815

Test Case 77.
Target:sqrt(abs(x1))
Skeleton:sqrt(abs(C**2*x1**2+2*C**2*x1+C**2))
Skeleton+LS:sqrt(abs(0.0031118541694899445**2*x1**2+2*0.7069681713459786*

Skeleton+LS:2*1.00137335046165*x1+2*0.26738504620109776+sqrt(abs(0.7810258442725446*x1+-0.00198869719706472))
Err:2.6452452973130295e-07

Test Case 109.
Target:0.06*x1**2+sin(x1+0.51)
Skeleton:sqrt(abs(C*x1+C))+sin(C*x1+C)
Skeleton+LS:sqrt(abs(0.5346089733666428*x1+-0.06100521149443685))+sin(0.5656488207011277*x1+2.2999020663713603)
Err:0.1462134059132685

Test Case 110.
Target:1.42*x1
Skeleton:C**2*x1**2+2*C**2*x1+C**2+sin(C*x1+C)
Skeleton+LS:-0.1340391462515006**2*x1**2+2*0.8450226943132388**2*x1+0.9998576295268908**2+sin(-0.19399677724118233*x1+-1.540873641440735)
Err:2.502286674743007e-06

Test Case 111.
Target:-0.04*x1**2+1.07*x1+0.26
Skeleton:4*C*x1+4*C
Skeleton+LS:4*0.24268530662207924*x1+4*0.07759000043050429
Err:0.01166300669145637

Test Case 112.
Target:sqrt(abs(x1+0.67))
Skeleton:sqrt(abs(C*x1+C))
Skeleton+LS:sqrt(abs(1.0006410613973957*x1+0.6691792634360204))
Err:3.930346808352507e-08

Test Case 113.
Target:2*x1+sqrt(abs(0.91*x1))
Skeleton:2*C*x1*sqrt(abs(C*x1+C))+2*C*sqrt(

Target:sin(1.31*x1)
Skeleton:2*C**2*x1**2+4*C**2*x1+2*C**2
Skeleton+LS:2*-0.26167293438660155**2*x1**2+4*-4.2204202009456774e-07**2*x1+2*0.6432217336145587**2
Err:6.809583855051186

Test Case 147.
Target:sin(1.72*x1-0.33)
Skeleton:C**2*x1**2+2*C**2*x1+C**2
Skeleton+LS:-0.8008605836306399**2*x1**2+2*0.8408378331421191**2*x1+-0.00016228732955647992**2
Err:75.95316795014593

Test Case 148.
Target:sqrt(abs(-0.39*x1))*sin(0.99*x1+0.27)
Skeleton:sin(C**2*x1**2+2*C**2*x1+C**2)
Skeleton+LS:sin(0.9992876002229593**2*x1**2+2*1.580999820677708**2*x1+-2.7870828648350677e-05**2)
Err:1.4867467043962748

Test Case 149.
Target:0.77*x1*sqrt(abs(x1))
Skeleton:C*x1*sqrt(abs(C*x1+C))+C*sqrt(abs(C*x1+C))
Skeleton+LS:0.8132072122950249*x1*sqrt(abs(0.8922636843351452*x1+0.011922497982198813))+-0.0038869610564110702*sqrt(abs(0.8370776356186646*x1+0.5387180416222944))
Err:4.05205548056391e-06

Test Case 150.
Target:0.65*x1+0.6
Skeleton:C*x1+C+sqrt(abs(C*x1+C))
Skeleton+LS:0.6508763834438953*x1+-0.3713418014628

...

Skeleton+LS:sqrt(abs(0.003292775712734414*x2+1.4913779063105135))
Err:0.03849940137487068

Test Case 723.
Target:x1*x3*sqrt(abs(x2))+0.1*x1-0.08*x3
Skeleton:C**3*x1**2*x2+C**3*x1**2+C**3*x1*x2**2+4*C**3*x1*x2+3*C**3*x1+C**3*x2**2+C**3*x2*x4+3*C**3*x2+C**3*x
Skeleton+LS:1.0**3*x1**2*x2+1.0**3*x1**2+1.0**3*x1*x2**2+4*1.0**3*x1*x2+3*1.0**3*x1+1.0**3*x2**2+1.0**3*x2*x4+3*1.0**3*x2+1.0**3*x
Err:61.04833127158389

Test Case 724.
Target:-0.22*x3-0.25
Skeleton:C*x3*sqrt(abs(C*x1+C))+C*sqrt(abs(C*x1+C))
Skeleton+LS:-0.11813057552679437*x3*sqrt(abs(0.0008382514233815477*x1+3.361082707191288))+-0.1514417979134246*sqrt(abs(0.00014762772731789276*x1+2.705023794910655))
Err:0.000253177715165284

Test Case 725.
Target:-0.97*x4
Skeleton:C**2*x3*x4+C**2*x3+C**2*x4+C**2+sin(C*x4+C)
Skeleton+LS:-0.016380377118046182**2*x3*x4+-0.0004700364356319698**2*x3+-0.9831244292749445**2*x4+-0.33132645005585615**2+sin(0.0014963775453027128*x4+6.3924162287419435)
Err:0.00027874501212464005

Test Case 726.
Target:-sin

Target:1.5*x3**2
Skeleton:C**4*x3**3+C**4*x3**3*x4+4*C**4*x3**2+2*C**4*x3*x4+5*C**4*x3+C**4*x4+2*C**4
Skeleton+LS:-1.102667583721407**4*x3**3+-0.038323788820379595**4*x3**3*x4+4*0.5967435184573533**4*x3**2+2*-0.060372684523393026**4*x3*x4+5*0.04880093817906329**4*x3+0.03114400745888985**4*x4+2*0.12427494134371589**4
Err:27.01694062725707

Test Case 754.
Target:sin(x1)
Skeleton:sqrt(abs(C*x1+C))*sin(C*x1+C)
Skeleton+LS:sqrt(abs(0.7242496097644125*x1+-0.031263969756887625))*sin(-0.822691784583702*x1+2.5340449870915003)
Err:0.585479081420591

Test Case 755.
Target:-0.19*x1*x2*x4
Skeleton:C**3*x1*x2*x4+C**3*x1*x2+C**3*x1*x4+C**3*x1+C**3*x2*x4+C**3*x2+C**3*x4+C**3
Skeleton+LS:-0.5666967730498411**3*x1*x2*x4+-0.08465663173429742**3*x1*x2+-0.06764568551208763**3*x1*x4+0.024361588103245166**3*x1+-0.0973459082058312**3*x2*x4+0.10414169247143076**3*x2+0.08847424004159689**3*x4+0.001450038575393317**3
Err:0.6207290027315262

Test Case 756.
Target:sqrt(abs(x1))+sqrt(abs(-0.84*x3))
Skeleton:2*C*x1+

Target:sin(0.95*x1+x4)
Skeleton:sin(C**2*x1*x4+C**2*x1+C**2*x4+C**2)
Skeleton+LS:sin(0.0005142195624742241**2*x1*x4+0.9766321129858024**2*x1+1.0001200226709948**2*x4+-0.03090907519335376**2)
Err:0.00015356193118696055

Test Case 785.
Target:sin(sqrt(abs(0.57*x2+0.56)))
Skeleton:sin(C*x1+C*x2+2*C)
Skeleton+LS:sin(0.0006394453195986833*x1+0.27599189068866153*x2+2*3.5271898908888297)
Err:0.014813046712226784

Test Case 786.
Target:-0.99*x2**2-0.83*x3
Skeleton:C**2*x2*x3+C**2*x2+C**2*x3+C**2
Skeleton+LS:-0.19496860109156927**2*x2*x3+-1.699415135225038**2*x2+-0.900385688834139**2*x3+1.2094837709185853**2
Err:101.30438615348292

Test Case 787.
Target:sqrt(abs(sin(0.81*x1)))
Skeleton:sqrt(abs(C*x1+C))*sin(C*x1+C)
Skeleton+LS:sqrt(abs(0.7978996571181991*x1+-5.474865922777652e-09))*sin(-0.3567508952708229*x1+1.6258141662836934)
Err:1.125595632528392

Test Case 788.
Target:0.15*x1*x3*sqrt(abs(x3+0.34))-0.78*x1+0.01*x3-0.05
Skeleton:C**2*x1*x3+C**2*x1+C**2*x3+C**2
Skeleton+LS:-0.14461228735937898

Target:-0.03*x1*x2-sin(0.44*x2-0.11)
Skeleton:C*x2*sqrt(abs(C*x1+C))+C*sqrt(abs(C*x1+C))
Skeleton+LS:-0.21858061563612044*x2*sqrt(abs(0.5607560496754624*x1+2.5853242808023857))+0.034408357519276594*sqrt(abs(0.09470053819726168*x1+2.4962921664532334))
Err:0.5909568389266351

Test Case 818.
Target:-0.58*x2+x3*sqrt(abs(0.97*x5))
Skeleton:C**3*x2*x3*x5+C**3*x2*x3+C**3*x2*x5+C**3*x2+C**3*x3*x5+C**3*x3+C**3*x5+C**3
Skeleton+LS:0.10342319770583222**3*x2*x3*x5+0.11636026398089587**3*x2*x3+-0.6491989806410896**3*x2*x5+-0.6319936876038014**3*x2+0.7809178562492094**3*x3*x5+0.7519160647233025**3*x3+-0.14845705293955733**3*x5+0.03536510789512157**3
Err:0.14243453099302048

Test Case 819.
Target:sqrt(abs(x2))-sin(0.91*x1-0.31)
Skeleton:C*x1*sqrt(abs(C*x2+C))+C*x3*sqrt(abs(C*x2+C))+2*C*sqrt(abs(C*x2+C))
Skeleton+LS:-0.2582515061879652*x1*sqrt(abs(-0.009967116383178041*x2+2.875578013935846))+0.019483085898485*x3*sqrt(abs(-1.6525091659780962*x2+5.129358336033952))+2*0.2718051995818024*sqrt(abs(3.432240

Skeleton+LS:sqrt(abs(-0.598219119660671*x3+1.789232415363184))*sin(0.028247037169389623*x1+0.7511402052448329)
Err:2.0247453820090153

Test Case 850.
Target:sin(0.3*x1**2-0.88*x1)
Skeleton:sin(C**2*x1*x2+C**2*x1+C**2*x2+C**2)
Skeleton+LS:sin(-0.18574612151987255**2*x1*x2+0.25132667127954256**2*x1+0.19916207453416646**2*x2+2.409158209601806**2)
Err:1.1645831027039442

Test Case 851.
Target:0.66*x1**2+0.6*x1*sqrt(abs(0.11*x3+0.2))
Skeleton:C**2*x1**2+C**2*x1*x3+3*C**2*x1+C**2*x3+2*C**2
Skeleton+LS:0.6337843567349033**2*x1**2+0.5009487513419925**2*x1*x3+3*-0.10281111481853152**2*x1+-0.3318935499585705**2*x3+2*0.30117034196914794**2
Err:0.03365969773875504

Test Case 852.
Target:0.52*x1
Skeleton:C**2*x1**2+2*C**2*x1+C**2
Skeleton+LS:-0.012784551222871823**2*x1**2+2*0.5112921074111934**2*x1+0.007908388695157389**2
Err:1.667876217187283e-05

Test Case 853.
Target:0.03*x2+sqrt(abs(0.68-0.87*x3))-0.92
Skeleton:sqrt(abs(C*x3+C))*sin(C*x5+C)
Skeleton+LS:sqrt(abs(0.014592445573005748*x3+-0.009776

In [ ]:
# plot the error frequency for model comparison
from matplotlib import pyplot as plt
num_eqns = len(resultDict[fName]['SymbolicGPT'])
num_vars = pconf.numberofVars

models = list(key for key in resultDict[fName].keys() if len(resultDict[fName][key])==num_eqns)
lists_of_error_scores = [resultDict[fName][key] for key in models if len(resultDict[fName][key])==num_eqns]
linestyles = ["-","dashdot","dotted","--"]

eps = 0.00001
y, x, _ = plt.hist([np.log([max(min(x+eps, 1e5),1e-5) for x in e]) for e in lists_of_error_scores],
                   label=models,
                   cumulative=True, 
                   histtype="step", 
                   bins=2000, 
                   density=True,
                   log=False)
y = np.expand_dims(y,0)
plt.figure(figsize=(15, 10))

for idx, m in enumerate(models): 
    plt.plot(x[:-1], 
           y[idx] * 100, 
           linestyle=linestyles[idx], 
           label=m)

plt.legend(loc="upper left")
plt.title("{} equations of {} variables - Benchmark".format(num_eqns, num_vars))
plt.xlabel("Log of Relative Mean Square Error")
plt.ylabel("Normalized Cumulative Frequency")

name = '{}.png'.format(fName.split('.txt')[0])
plt.savefig(name)

In [ ]:
# benchmarks
import csv
benchmarkPath = './benchmark/dsr-benchmark-data/'
dataPoints = glob.glob(benchmarkPath+'*.csv')
NGUYEN2Eq = {
           '1':'x1**3+x1**2+x1',
           '2':'x1**4+x1**3+x1**2+x1',
           '3':'x1**5+x1**4+x1**3+x1**2+x1',
           '4':'x1**6+x1**5+x1**4+x1**3+x1**2+x1',
           '5':'sin(x1**2)*cos(x1)-1',
           '6':'sin(x1)+sin(x1+x1**2)',
           '7':'log(x1+1)+log(x1**2+1)',
           '8':'sqrt(x1)',
           '9':'sin(x1)+sin(x2**2)',
           '10':'2*sin(x1)*cos(x2)',
           '11':'x1**x2',
           '12':'x1**4-x1**3+x2**2/2-x2',}
for dataPoint in dataPoints: 
    key = dataPoint.split('\\')[-1].split('Nguyen-')[-1].split('_')[0]
    if not key in NGUYEN2Eq.keys():
        continue
    target = NGUYEN2Eq[key]
    with open(dataPoint, newline='\n') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='|')
        pointsList = []
        pointsListTest = []
        for i, c in enumerate(reader):
            p = ([eval(x) for x in c[:-1]],eval(c[-1]))
            if i < numPoints:
                pointsList.append(p)
            else:
                pointsListTest.append(p)
                
        #pointsList = [([eval(x) for x in p[:-1]],eval(p[-1])) for i,p in enumerate(reader) if i < numPoints]
        #pointsListTest = [([eval(x) for x in p[:-1]],eval(p[-1])) for i,p in enumerate(reader) if i >= numPoints]
        
        # initialized the input variable with start token <
        inputs = torch.tensor([[train_dataset.stoi['<']]]).to(trainer.device)
        
        # extract points from the input sequence
        points = torch.zeros(numVars+numYs, numPoints)
        for idx, xy in enumerate(pointsList):
            x = xy[0] + [0]*(max(numVars-len(xy[0]),0)) # padding
            y = [xy[1]] if type(xy[1])== float else xy[1]
            y = y + [0]*(max(numYs-len(y),0)) # padding
            p = x+y # because it is only one point 
            p = torch.tensor(p)
            #replace nan and inf
            p = torch.nan_to_num(p, nan=0.0, 
                                 posinf=train_dataset.threshold[1], 
                                 neginf=train_dataset.threshold[0])
            p[p>train_dataset.threshold[1]] = train_dataset.threshold[1] # clip the upper bound
            p[p<train_dataset.threshold[0]] = train_dataset.threshold[0] # clip the lower bound
            points[:,idx] = p
            
        points = points.unsqueeze(0).to(trainer.device)
        outputsHat = sample(model, inputs, blockSize, points=points,
                      temperature=1.0, sample=False, 
                      top_k=40)[0]
        
        # filter out predicted
        predicted = ''.join([train_dataset.itos[int(i)] for i in outputsHat])
        predicted = predicted.strip(train_dataset.paddingToken).split('>')
        predicted = predicted[0] if len(predicted[0])>=1 else predicted[1]
        predicted = predicted.strip('<').strip(">")
        
        # extract points from the input sequence
        pointsTest = torch.zeros(numVars+numYs, numPoints).numpy()
        for idx, xy in enumerate(pointsListTest):
            x = xy[0] + [0]*(max(numVars-len(xy[0]),0)) # padding
            y = [xy[1]] if type(xy[1])== float else xy[1]
            y = y + [0]*(max(numYs-len(y),0)) # padding
            p = x+y # because it is only one point 
            p = torch.tensor(p)
            #replace nan and inf
            p = torch.nan_to_num(p, nan=0.0, 
                                 posinf=train_dataset.threshold[1], 
                                 neginf=train_dataset.threshold[0])
            p[p>train_dataset.threshold[1]] = train_dataset.threshold[1] # clip the upper bound
            p[p<train_dataset.threshold[0]] = train_dataset.threshold[0] # clip the lower bound
            pointsTest[:,idx] = p
        
        Ys = [] 
        Yhats = []
        for xs in pointsTest[:-1,:].T:
            try:
                eqTmp = target + '' # copy eq
                eqTmp = eqTmp.replace(' ','')
                eqTmp = eqTmp.replace('\n','')
                for i,x in enumerate(xs):
                    # replace xi with the value in the eq
                    eqTmp = eqTmp.replace('x{}'.format(i+1), str(x))
                    if ',' in eqTmp:
                        assert 'There is a , in the equation!'
                YEval = eval(eqTmp)
                YEval = 0 if np.isnan(YEval) else YEval
                YEval = 100 if np.isinf(YEval) else YEval
            except:
                YEval = 100 #TODO: Maybe I have to punish the model for each wrong template not for each point
            Ys.append(YEval)
            try:
                eqTmp = predicted + '' # copy eq
                eqTmp = eqTmp.replace(' ','')
                eqTmp = eqTmp.replace('\n','')
                for i,x in enumerate(xs):
                    # replace xi with the value in the eq
                    eqTmp = eqTmp.replace('x{}'.format(i+1), str(x))
                    if ',' in eqTmp:
                        assert 'There is a , in the equation!'
                Yhat = eval(eqTmp)
                Yhat = 0 if np.isnan(Yhat) else Yhat
                Yhat = 100 if np.isinf(Yhat) else Yhat
            except:
                Yhat = 100
            Yhats.append(Yhat)
        err = relativeErr(Ys,Yhats)
        
        print('NGUYEN-{} --> Target:{}\nPredicted:{}\nErr:{}\n'.format(key, target, predicted, err))